# Tutorial 6 - External Function Interfacing

In [1]:
# !pip install taskgen-ai

In [28]:
# Set up API key and do the necessary imports
import os
from taskgen import *
import random

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Adding Python Function directly to Function
- You should write a docstring that describes the function and contains all the input variable names that are not `args`, `kwargs` or `shared_variables`
- typing for inputs and outputs will be automatically converted to TaskGen `Function` format

In [3]:
from typing import List, Dict

def add_number_to_list(num1: int, num_list: List[int], *args, **kwargs) -> List[int]:
    ''' Appends num1 to num_list '''
    num_list.append(num1)
    return num_list

In [4]:
fn = Function(external_fn = add_number_to_list)

In [5]:
print(fn)

Description:  Appends <num1: int> to <num_list: list[int]> 
Input: ['num1', 'num_list']
Output: {'output_1': 'list[int]'}



# Adding Python Function directly to Agent
- We can also assign the Python function directly to an Agent and it will automatically convert it to `Function` class

In [6]:
agent = Agent('Math Whiz', 'Does Math Calculations').assign_functions([add_number_to_list])

In [7]:
agent.list_functions()

["Name: use_llm\nDescription: Used only when no other function can do the task\nInput: []\nOutput: {'Output': 'Output of LLM'}\n",
 'Name: end_task\nDescription: Use only when task is completed\nInput: []\nOutput: {}\n',
 "Name: add_number_to_list\nDescription:  Appends <num1: int> to <num_list: list[int]> \nInput: ['num1', 'num_list']\nOutput: {'output_1': 'list[int]'}\n"]

In [8]:
agent.run('Append 5 to [2, 4]')

Subtask identified: Append the number 5 to the list [2, 4]
Calling function add_number_to_list with parameters {'num1': 5, 'num_list': [2, 4]}
> {'output_1': [2, 4, 5]}

Subtask identified: Append 5 to [2, 4]
Calling function add_number_to_list with parameters {'num1': 5, 'num_list': [2, 4]}
> {'output_1': [2, 4, 5]}

Task completed successfully!



[{'output_1': [2, 4, 5]}, {'output_1': [2, 4, 5]}]

# CrewAI Structured Tools Interface with TaskGen

In [7]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [54]:
from crewai_tools import ScrapeWebsiteTool

In [55]:
# just wrap the external tool within a function
def scrape_website_tool(website_url: str) -> str:
    ''' Scrapes data from website_url '''
    # import the tool
    from crewai_tools import ScrapeWebsiteTool
    
    # initialise the tool
    docs_scrape_tool = ScrapeWebsiteTool(
        website_url=website_url)
    
    # run the tool
    return docs_scrape_tool.run()

In [56]:
agent = Agent('Website summariser', 'Scrapes and then summarises a website to the user').assign_functions([scrape_website_tool])

In [57]:
agent.run('Summarise "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"')

Subtask identified: Scrape the website "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
Calling function scrape_website_tool with parameters {'website_url': 'https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/'}
Using Tool: Read website content
> {'output_1': 'Assembling and Activating Your CrewAI Team - crewAI\n Skip to content\n crewAI\n Assembling and Activating Your CrewAI Team\n crewAI\n crewAI\n crewAI\n Home\n Core Concepts\n Core Concepts\n Agents\n Tasks\n Tools\n Processes\n Crews\n Collaboration\n Memory\n How to Guides\n How to Guides\n Installing CrewAI\n Getting Started\n Getting Started\n Table of contents\n Introduction\n Step 0: Installation\n Step 1: Assemble Your Agents\n Step 2: Define the Tasks\n Step 3: Form the Crew\n Step 4: Kick It Off\n Conclusion\n Create Custom Tools\n Using Sequential Process\n Using Hierarchical Process\n Connecting to any LLM\n Customizing Agents\n Human Input on Execution\n Agent Monitoring with AgentOps\n To

[{'output_1': 'Assembling and Activating Your CrewAI Team - crewAI\n Skip to content\n crewAI\n Assembling and Activating Your CrewAI Team\n crewAI\n crewAI\n crewAI\n Home\n Core Concepts\n Core Concepts\n Agents\n Tasks\n Tools\n Processes\n Crews\n Collaboration\n Memory\n How to Guides\n How to Guides\n Installing CrewAI\n Getting Started\n Getting Started\n Table of contents\n Introduction\n Step 0: Installation\n Step 1: Assemble Your Agents\n Step 2: Define the Tasks\n Step 3: Form the Crew\n Step 4: Kick It Off\n Conclusion\n Create Custom Tools\n Using Sequential Process\n Using Hierarchical Process\n Connecting to any LLM\n Customizing Agents\n Human Input on Execution\n Agent Monitoring with AgentOps\n Tools Docs\n Tools Docs\n Google Serper Search\n Browserbase Web Loader\n Scrape Website\n Directory Read\n File Read\n Selenium Scraper\n Directory RAG Search\n PDF RAG Search\n TXT RAG Search\n CSV RAG Search\n XML RAG Search\n JSON RAG Search\n Docx Rag Search\n MDX RAG Sea

In [58]:
agent.reply_user('What are the key imports needed for CrewAI?')

The key imports needed for CrewAI include importing the Agent class and tools from crewai and crewai_tools packages, as well as importing Task, Crew, and Process from crewai. Additionally, importing necessary packages like os for setting environment variables is required for configuring agents and tasks.


'The key imports needed for CrewAI include importing the Agent class and tools from crewai and crewai_tools packages, as well as importing Task, Crew, and Process from crewai. Additionally, importing necessary packages like os for setting environment variables is required for configuring agents and tasks.'

In [59]:
agent.reply_user('What version of packages are they compatible with?')

CrewAI is compatible with Python versions >=3.10 and <=3.13. The installation step mentions the compatibility with Python >=3.10,<=3.13, indicating that the CrewAI package is designed to work with these specific versions of Python.


'CrewAI is compatible with Python versions >=3.10 and <=3.13. The installation step mentions the compatibility with Python >=3.10,<=3.13, indicating that the CrewAI package is designed to work with these specific versions of Python.'

# LangChain Structured Tools Interface with TaskGen

In [9]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [23]:
from taskgen import Agent
agent = Agent('Math Whiz', 'Does calculations').assign_functions([add.func, multiply.func])

In [24]:
agent.list_functions()

["Name: use_llm\nDescription: Used only when no other function can do the task\nInput: []\nOutput: {'Output': 'Output of LLM'}\n",
 'Name: end_task\nDescription: Use only when task is completed\nInput: []\nOutput: {}\n',
 "Name: add\nDescription: Adds <a: int> and <b: int>.\nInput: ['a', 'b']\nOutput: {'output_1': 'int'}\n",
 "Name: multiply\nDescription: Multiplies <a: int> and <b: int>.\nInput: ['a', 'b']\nOutput: {'output_1': 'int'}\n"]

In [25]:
agent.run('Calculate 3*5+3')

Subtask identified: Multiply 3 and 5
Calling function multiply with parameters {'a': 3, 'b': 5}
> {'output_1': 15}

Subtask identified: Add the result of the multiplication to 3
Calling function add with parameters {'a': 15, 'b': 3}
> {'output_1': 18}

Task completed successfully!



[{'output_1': 15}, {'output_1': 18}]

# LangChain Community Tools Interface with TaskGen

In [13]:
def wikipedia_tool(search_query: str) -> str:
    ''' Uses search_query and returns text from wikipedia '''
    from langchain.tools import WikipediaQueryRun
    from langchain.utilities import WikipediaAPIWrapper
    from langchain.agents import Tool

    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return wikipedia.run(search_query)

In [14]:
agent = Agent('Wiki Agent', 'Searches Wiki').assign_functions(wikipedia_tool)

In [15]:
agent.run('Get me information on LangChain', num_subtasks = 1)

Subtask identified: Use the equipped function wikipedia_tool to search for LangChain on Wikipedia
Calling function wikipedia_tool with parameters {'search_query': 'LangChain'}
> {'output_1': 'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Prompt engineering\nSummary: Prompt engineering is the process of structuring an instruction that can be interpreted and understood by a generative AI model. A prompt is natural language text describing the task that an AI should perform.\nA prompt for a text-to-text language model can be a query such as "what is Fermat\'s little theorem?", a command such as "write a poem about leaves falling", or a longer statement including context, instructions, and c

[{'output_1': 'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Prompt engineering\nSummary: Prompt engineering is the process of structuring an instruction that can be interpreted and understood by a generative AI model. A prompt is natural language text describing the task that an AI should perform.\nA prompt for a text-to-text language model can be a query such as "what is Fermat\'s little theorem?", a command such as "write a poem about leaves falling", or a longer statement including context, instructions, and conversation history. Prompt engineering may involve phrasing a query, specifying a style, providing relevant context or assigning a role to the AI such as "Act as a native Frenc

# LlamaIndex Function Interface with TaskGen

In [16]:
from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str:
    """Usfeful for getting the weather for a given location."""
    return 'Sunny'
    ...

tool = FunctionTool.from_defaults(
    get_weather,
    # async_fn=aget_weather,  # optional!
)

In [17]:
agent = Agent('Weather Agent', 'Returns the Weather').assign_functions(tool.fn)

In [18]:
agent.list_functions()

["Name: use_llm\nDescription: Used only when no other function can do the task\nInput: []\nOutput: {'Output': 'Output of LLM'}\n",
 'Name: end_task\nDescription: Use only when task is completed\nInput: []\nOutput: {}\n',
 "Name: get_weather\nDescription: Usfeful for getting the weather for a given <location: str>.\nInput: ['location']\nOutput: {'output_1': 'str'}\n"]

In [19]:
agent.run('What is the weather in Miami?')

Subtask identified: Use the get_weather function with "Miami" as the location input
Calling function get_weather with parameters {'location': 'Miami'}
> {'output_1': 'Sunny'}

Task completed successfully!



[{'output_1': 'Sunny'}]